In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Sampler, BatchSampler, Dataset

import optuna

import tables
import numpy as np
from sklearn.model_selection import train_test_split

import logging
import sys

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Load data

In [2]:
class ExpertDataset(Dataset):
    def __init__(self, images, actions):
        self._images = images
        self._actions = actions

    def __getitem__(self, index):
        return (self._images[index], self._actions[index])

    def __len__(self):
        return len(self._images)


class RandomBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        self.batch_size = batch_size
        self.dataset_length = len(dataset)
        self.n_batches = self.dataset_length / self.batch_size
        self.batch_ids = torch.randperm(int(self.n_batches))

    def __len__(self):
        return self.batch_size

    def __iter__(self):
        for id in self.batch_ids:
            idx = torch.arange(id * self.batch_size, (id + 1) * self.batch_size)
            for index in idx:
                yield int(index)
        if int(self.n_batches) < self.n_batches:
            idx = torch.arange(int(self.n_batches) * self.batch_size, self.dataset_length)
            for index in idx:
                yield int(index)


def fast_loader(dataset, batch_size=32, drop_last=False, transforms=None):
    return DataLoader(
        dataset, batch_size=None,
        sampler=BatchSampler(RandomBatchSampler(dataset, batch_size), batch_size=batch_size, drop_last=drop_last)
    )

In [3]:
hdf5_file = tables.open_file('tmnf_1os.hdf5', mode="r")
images = np.moveaxis(hdf5_file.root.images[:], 3, 1)
actions = hdf5_file.root.actions[:]
hdf5_file.close()

size = images.shape[0]
fold = 0.3
rng = np.random.default_rng()
indexes = rng.choice(size, size=int(fold * size), replace=False)

images = images[indexes]
actions = actions[indexes]

images = torch.tensor(images / 255, dtype=torch.float32)
actions = torch.tensor(actions, dtype=torch.float32)

train_images, test_images, train_actions, test_actions = train_test_split(
    images, actions, test_size=0.15, random_state=42
)

In [4]:
train_dataset = ExpertDataset(train_images, train_actions)
test_dataset = ExpertDataset(test_images, test_actions)

# NN Model

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, conv_layers_types: list[str], conv_layers_outputs: list[int], kernel_sizes: list[int], strides: list[int]):
        super(NeuralNetwork, self).__init__()

        inputs = 4
        outputs = 4
        convw, convh = 53, 150
        layers = []

        for layer_type, layer_outputs, kernel_size, stride in zip(conv_layers_types, conv_layers_outputs, kernel_sizes, strides):
            layers.append(nn.Conv2d(inputs, layer_outputs, kernel_size=kernel_size, stride=stride, padding=0))
            if layer_type == 'relu':
                layers.append(nn.ReLU())
            elif layer_type == 'tanh':
                layers.append(nn.Tanh())
            elif layer_type == 'sigmoid':
                layers.append(nn.Sigmoid())
            inputs = layer_outputs
            convw = (convw - kernel_size) // stride + 1
            convh = (convh - kernel_size) // stride + 1

        layers.append(nn.Flatten())
        layers.append(nn.Linear(inputs * convw * convh, outputs))
        layers.append(nn.Sigmoid())

        self.layers_sequence = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.layers_sequence(x)
        return logits

# Training

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0., 0.
    threshod = 0.5
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred = pred > threshod
            correct += (pred == y).type(torch.float).sum().item() == len(y[0])
            # correct += (pred == y).type(torch.float).sum().item() / len(y[0])
    test_loss /= num_batches
    correct /= size
    # print(f"test accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}")
    return correct * 100, test_loss

# Optuna

In [7]:
def objective(trial):

    epochs = 10

    # layer_1 = trial.suggest_categorical("layer_1", ["relu", "tanh", "sigmoid"])
    # layer_2 = trial.suggest_categorical("layer_2", ["relu", "tanh", "sigmoid"])

    # units_1 = trial.suggest_int("units_1", 300, 600, step=25)
    # units_2 = trial.suggest_int("units_2", 150, 350, step=25)

    layer_1 = "relu"
    layer_2 = "tanh"
    layer_3 = "tanh"

    outs_1 = 32
    outs_2 = 64
    outs_3 = 64

    kernel_size_1 = 8
    kernel_size_2 = 4
    kernel_size_3 = 3
    
    stride_1 = 4
    stride_2 = 2
    stride_3 = 1

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 512, log=True)

    layers = [layer_1, layer_2, layer_3]
    layers_out = [outs_1, outs_2, outs_3]
    kernel_sizes = [kernel_size_1, kernel_size_2, kernel_size_3]
    strides = [stride_1, stride_2, stride_3]

    model = NeuralNetwork(layers, layers_out, kernel_sizes, strides).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_dataloader = fast_loader(train_dataset, batch_size=batch_size)
    test_dataloader = fast_loader(test_dataset, batch_size=1)


    global best
    for epoch in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        acc, loss = test(test_dataloader, model, loss_fn)
        
        if acc > best:
            best = acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'acc': acc
                }, f'{trial.number}-{epoch}-acc{int(acc)}')
    return best
    

# Test

In [8]:
best = 0

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

trial = study.best_trial

print("Best trial:")
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-06-28 20:14:10,398] A new study created in memory with name: no-name-a794e7a9-e12a-4a3f-968d-f071d4335693


A new study created in memory with name: no-name-a794e7a9-e12a-4a3f-968d-f071d4335693


[I 2022-06-28 20:19:15,970] Trial 0 finished with value: 34.40673767649244 and parameters: {'learning_rate': 0.0012857756342430254, 'batch_size': 33}. Best is trial 0 with value: 34.40673767649244.


Trial 0 finished with value: 34.40673767649244 and parameters: {'learning_rate': 0.0012857756342430254, 'batch_size': 33}. Best is trial 0 with value: 34.40673767649244.


[I 2022-06-28 20:23:47,897] Trial 1 finished with value: 34.40673767649244 and parameters: {'learning_rate': 0.006147973187478454, 'batch_size': 67}. Best is trial 0 with value: 34.40673767649244.


...

[I 2022-06-28 20:54:01,792] Trial 8 finished with value: 57.666584097101804 and parameters: {'learning_rate': 0.005737614525998634, 'batch_size': 232}. Best is trial 5 with value: 57.666584097101804.


Trial 8 finished with value: 57.666584097101804 and parameters: {'learning_rate': 0.005737614525998634, 'batch_size': 232}. Best is trial 5 with value: 57.666584097101804.


[I 2022-06-28 20:58:33,675] Trial 9 finished with value: 57.666584097101804 and parameters: {'learning_rate': 0.004440522787669779, 'batch_size': 142}. Best is trial 5 with value: 57.666584097101804.


Trial 9 finished with value: 57.666584097101804 and parameters: {'learning_rate': 0.004440522787669779, 'batch_size': 142}. Best is trial 5 with value: 57.666584097101804.
Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10
Best trial:
  Value:  57.666584097101804
  Params: 
    learning_rate: 0.0016062304850273166
    batch_size: 411


C:\ProgramData\Anaconda3\envs\py310\lib\site-packages\optuna\structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)
